In [1]:
import os
import torch

# Check for the availability of CUDA-enabled GPU
if torch.cuda.is_available():
    # Set device to GPU
    torch.cuda.set_device(0)
    device = torch.device("cuda")
    print("Switched to GPU")
else:
    # Use CPU if no GPU is available
    device = torch.device("cpu")
    print("No GPU available, using CPU")

print("Using device:", device)


No GPU available, using CPU
Using device: cpu


In [3]:
from elasticsearch import Elasticsearch

In [4]:
es=Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","elastic"),
    ca_certs="elasticsearch-8.14.1\config\certs\http_ca.crt"
)
es.ping()

<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\c'
C:\Users\dell\AppData\Local\Temp\ipykernel_9156\2381342347.py:4: SyntaxWarning: invalid escape sequence '\c'
  ca_certs="elasticsearch-8.14.1\config\certs\http_ca.crt"


True

DATA PREPARATION

In [4]:
import pandas as pd
df=pd.read_csv("data2.csv")
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [5]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           11597
                                                                                   True              894
Name: count, dtype: int64

In [6]:
df.fillna("None")

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White
...,...,...,...,...,...,...,...,...
12486,10262843,Pepe Jeans Men Black Hammock Slim Fit Low-Rise...,Pepe Jeans,Men,1299,7,"Black dark wash 5-pocket low-rise jeans, clean...",Black
12487,10261721,Mochi Women Gold-Toned Solid Heels,Mochi,Women,1990,5,"A pair of gold-toned open toe heels, has regul...",Gold
12488,10261607,612 league Girls Navy Blue & White Printed Reg...,612 league,Girls,602,4,Navy Blue and White printed mid-rise denim sho...,Blue
12489,10266621,Bvlgari Men Aqva Pour Homme Marine Eau de Toil...,Bvlgari,Men,8950,2,Bvlgari Men Aqva Pour Homme Marine Eau de Toil...,None


In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
# Convert all descriptions to string and then apply the encoding
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))

In [25]:
df["ProductNameVector"] = df["ProductName"].apply(lambda x: model.encode(x))


In [109]:
df["BrandVector"] = df["ProductBrand"].apply(lambda x: model.encode(x))
df["ColorVector"] = df["PrimaryColor"].apply(lambda x: model.encode(x))

TypeError: 'float' object is not subscriptable

In [26]:
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector,ProductNameVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645722, -0.002634206, -0.0035884539, 0.0...","[0.023021469, -0.005015909, -0.011468837, 0.02..."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.024660705, -0.028755426, -0.02033245, 0.03...","[-0.0096334135, -0.026905974, -0.0055635665, 0..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.046943333, 0.081827894, 0.04833522, -0.000...","[-0.014643625, 0.057657227, 0.022244366, -0.01..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.015098721, -0.010285556, 0.009487335, -0.0...","[-0.00043136842, -0.011626249, -0.0070335884, ..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[-0.017746545, 0.0062095397, 0.021813968, 0.02...","[-0.025429703, -0.0009659374, 0.021511225, 0.0..."


Index Creation

In [9]:
indexMapping = {
    "properties":{
        "ProductID":{
            "type":"long"
        },
        "ProductName":{
            "type":"text"
        },
        "ProductBrand":{
            "type":"text"
        },
        "Gender":{
            "type":"text"
        },
        "Price (INR)":{
            "type":"long"
        },
        "NumImages":{
            "type":"long"
        },
        "Description":{
            "type":"text"
        },
        "PrimaryColor":{
            "type":"text"
        },
        "DescriptionVector":{
            "type":"dense_vector",
            "dims": 768,
            "index":True,
            "similarity": "l2_norm"
        }

    }
}

In [5]:
es.ping()

True

In [29]:
from indexMapping import indexMapping
index_name = "all_products"

# Check if the index exists
if es.indices.exists(index=index_name):
    # Delete the index if it exists
    es.indices.delete(index=index_name)

# Create the index with the new mapping
es.indices.create(index=index_name, mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

Ingest the data

In [30]:
record_list=df.to_dict("records")

In [31]:
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)

BadRequestError(400, 'document_parsing_exception', '[1:209] failed to parse: [1:227] Non-standard token \'NaN\': enable `JsonReadFeature.ALLOW_NON_NUMERIC_NUMBERS` to allow\n at [Source: (byte[])"{"ProductID":10001989,"ProductName":"Police Men To Be The King Eau De Toilette 125 ml","ProductBrand":"Police","Gender":"Men","Price (INR)":2695,"NumImages":5,"Description":"To Be The King Eau De Toilette\xa0","PrimaryColor":NaN,"DescriptionVector":[0.1043720543384552,0.04333436116576195,0.002058120910078287,0.01321625616401434,-0.025695253163576126,-0.0104330163449049,-0.10370036214590073,-0.005518186371773481,0.023120559751987457,0.005819679703563452,0.08082059770822525,-0.07898954302072525,-0.0"[truncated 32237 bytes]; line: 1, column: 227]')
BadRequestError(400, 'document_parsing_exception', '[1:300] failed to parse: [1:318] Non-standard token \'NaN\': enable `JsonReadFeature.ALLOW_NON_NUMERIC_NUMBERS` to allow\n at [Source: (byte[])"{"ProductID":10001265,"ProductName":"Michael Kors Women 

In [6]:
import numpy as np
def normalize_vector(vector):
    norm = np.linalg.norm(vector)
    if norm == 0: 
        return vector
    return vector / norm

In [7]:
es.count(index="all_products")

ObjectApiResponse({'count': 11597, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [11]:
input_keyword = "p"
vector_of_input = normalize_vector(model.encode(input_keyword))

In [12]:
vector_of_input

array([-5.23429923e-02,  5.58018796e-02,  2.98608169e-02, -3.05093993e-02,
       -2.08054204e-02, -9.29395109e-03, -1.41351419e-02,  5.00216223e-02,
       -4.25371900e-02, -2.25708839e-02,  2.35863607e-02,  3.61492820e-02,
        3.19983512e-02,  7.36519694e-02,  2.09540091e-02, -9.96233616e-03,
       -1.41097903e-02, -1.07592158e-03,  8.46140385e-02, -1.46411695e-02,
       -3.55583653e-02,  6.67535048e-03, -4.79257014e-03, -3.12233763e-03,
        1.45823220e-02, -9.30800736e-02, -3.24297696e-03,  3.69053297e-02,
       -4.20159325e-02,  4.91466969e-02,  2.51276810e-02,  8.86723213e-03,
       -2.28830911e-02, -5.38514555e-02,  1.65481799e-06, -3.69789675e-02,
        5.94456494e-03, -3.93538140e-02,  3.04362811e-02, -1.84808616e-02,
       -4.07486521e-02,  5.08695468e-03,  1.35546387e-03,  4.82289232e-02,
       -3.19495611e-02, -6.24145120e-02,  4.12963107e-02,  5.24583347e-02,
       -3.17343771e-02, -2.20335592e-02, -1.26383947e-02, -3.65602821e-02,
       -5.82423322e-02, -

In [13]:
knn_query_description = {
    "field": "DescriptionVector",
    "query_vector": vector_of_input,
    "k": 10,
    "num_candidates": 10000
}

# Perform k-NN search on ProductNameVector
knn_query_productname = {
    "field": "ProductNameVector",
    "query_vector": vector_of_input,
    "k": 20,
    "num_candidates": 10000
}


In [14]:
res_description = es.knn_search(index="all_products", knn=knn_query_description, _source=["ProductName", "Description"])
res_productname = es.knn_search(index="all_products", knn=knn_query_productname, _source=["ProductName", "Description"])
# Combine results
combined_hits = res_description["hits"]["hits"] + res_productname["hits"]["hits"]

# Calculate cosine similarity between input keyword vector and results vectors
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


C:\Users\dell\AppData\Local\Temp\ipykernel_9156\2967061899.py:1: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res_description = es.knn_search(index="all_products", knn=knn_query_description, _source=["ProductName", "Description"])
C:\Users\dell\AppData\Local\Temp\ipykernel_9156\2967061899.py:2: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res_productname = es.knn_search(index="all_products", knn=knn_query_productname, _source=["ProductName", "Description"])


In [15]:
input_tokens = set(input_keyword.lower().split())

threshold = 0.7  # Adjust this threshold based on desired similarity
filtered_hits = [
    hit for hit in combined_hits 
    if cosine_similarity(vector_of_input, model.encode(hit["_source"]["ProductName"])) > threshold and
    input_tokens & set(hit["_source"]["ProductName"].lower().split())
]



In [16]:
# Sort combined results by combined score (cosine similarity + Elasticsearch score)
filtered_hits = sorted(filtered_hits, key=lambda x: (0.5 * cosine_similarity(vector_of_input, model.encode(x["_source"]["ProductName"])) + 0.5 * x["_score"]), reverse=True)

for hit in filtered_hits[:20]:
    print(f"ProductName: {hit['_source']['ProductName']}, Description: {hit['_source']['Description']}, Score: {hit['_score']}")

ProductName: Slub Blue Washed Slim Jeans, Description: A pair of blue 5-pocket mid-rise jeans, lightly washed, has whiskers, a zip fly with a button closure, and a waistband with belt loops, Score: 0.87491775
ProductName: Slub Blue Washed Slim Jeans, Description: A pair of blue 5-pocket mid-rise jeans, lightly washed and lightly distressed, has whiskers and chevrons, a button fly closure, and a waistband with belt loopsOur stylist has paired these jeans with a beltThese jeans will not come with a belt, Score: 0.87491775
ProductName: Pepe Jeans Boys Blue Washed Mid-Rise Denim Shorts, Description: Blue washed mid-rise denim shorts, has an elasticated waistband, drawstring closure and four pockets, Score: 0.869617
ProductName: Pepe Jeans Boys Blue Washed Mid-Rise Denim Shorts, Description: Blue washed mid-rise denim shorts, has an elasticated waistband, drawstring closure and five pockets, Score: 0.869617
ProductName: Pepe Jeans Boys Blue Washed Denim Shorts, Description: Blue washed mid-